In [1]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer


import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional

import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")


C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

In [2]:
from Utilities.new_models import *
import utils_gym
import env_model_class_2


from board_env import *



%load_ext autoreload
%autoreload 2

In [3]:
from IPython.display import clear_output, display

In [26]:
class Trainer:
    """
    ## Trainer
    """

    def __init__(self, Args
                 ):
        # #### Configurations

        self.args = Args()#tyro.cli(Args)
        self.args.batch_size = int(self.args.num_envs * self.args.num_steps)
        self.args.minibatch_size = int(self.args.batch_size // self.args.num_minibatches)
        self.args.num_iterations = self.args.total_timesteps // self.args.batch_size
        self.run_name = f"{self.args.env_id}__{self.args.exp_name}__{self.args.seed}__{int(time.time())}"

        

        TB_log = True
        if TB_log:    
            self.writer = SummaryWriter(f"runs/{self.run_name}")
            self.writer.add_text(
                "hyperparameters",
                "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(self.args).items()])),
            )
        
        # TRY NOT TO MODIFY: seeding
        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        torch.backends.cudnn.deterministic = self.args.torch_deterministic
        
        self.device = torch.device("cuda" if torch.cuda.is_available() and self.args.cuda else "cpu")
        
        
        self.playe_r = 1#"agent_1" #
        
        self.num_steps = 60000#1000000
        self.action_shape = (2,)

        self.env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                            agent_count  = 3,use_placement_perc=True,render_=False)

        self.env = env_risk(**self.env_config)
        
        self.env.reset(seed=42)



        
        sample_obs = self.obs_converter(torch.tensor(self.env.last()[0]['observation']),num_classes = 4)
        
        self.ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
        self.action_mask_shape = self.env.observation_space(self.playe_r)['action_mask'].shape
        self.total_agents  = len(self.env.possible_agents)
        self.total_phases = len(self.env.phases)
        #self.device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")
        
        
        
        
        
        self.agent_list = list(self.env.possible_agents)
        
        
        
        self.num_episodes = 10
        self.args.gamma = self.gam = 0.99
        self.args.minibatch_size = 128        
        self.the_hero_agent = 1

        
        self.qnet_config_dict = dict(action_space = self.env.action_space(self.playe_r
                                                                         ).shape[0],
                                    ob_space=np.prod(self.ob_space_shape
                                                    )+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents -1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        self.actor_config_dict =  dict(env=self.env,
                        action_space = self.env.observation_space(self.playe_r)['action_mask'].shape[0],
                        ob_space=np.prod(self.ob_space_shape)+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents-1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        
        
        
        
        self.actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        self.qf1 = QNetwork(**self.qnet_config_dict).to(self.device)
        self.qf1_target = QNetwork(**self.qnet_config_dict).to(self.device)
        self.target_actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        
        
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.qf1_target.load_state_dict(self.qf1.state_dict())
        self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
        self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

        

        

    def obs_converter(self,  data, num_classes = 4, col =0 ):

        if col != None:
            return torch.concat((nn.functional.one_hot(data[:,col].detach().long(), 
                                                        num_classes = num_classes),
                                      data[:,~col,None]
                                ),axis=1
                               )[:,1:].to(self.device)
    
    def map_agent_phase_hot(self, data,num_classes = 3):
        return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:].to(self.device)
    
    def map_agent_phase_vector(self, data,num_classes = 3):
        return nn.functional.one_hot(data[:,0].long(), 
                                                            num_classes = num_classes)[:,1:].to(self.device)


    def train_loop_init(self):
        self.gamma_t = {i:0 for i in self.env.possible_agents}
        self.num_iterations = 500
        self.episode_time_lim = 5000
        self.draw_count = 0
        self.first_count = 0
        self.second_count = 0
        self.third_count = 0
        self.third_count_draw = 0
        self.start_time = time.time()
        self.global_step = 0
        #self.faulting_player = ""
        
    
    def run_training_loop(self):
        """
        ### Run training loop
        """

        # last 100 episode information
        #tracker.set_queue('reward', 100, True)
        #tracker.set_queue('length', 100, True)

        obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
        actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
        action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
        current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
        current_phase = torch.zeros((self.num_steps,1)).to(self.device)
        current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
        logprobs = torch.zeros((self.num_steps, )).to(self.device)
        rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        values = torch.zeros((self.num_steps,  )).to(self.device)
        episodes = torch.ones((self.num_steps, )).to(self.device)*-1
        t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)



        rb = ReplayBuffer(
                self.args.buffer_size,
                Box(low =0, high=2000, shape =(self.qnet_config_dict['ob_space']+1,), dtype=np.float32),
                Box(low =0, high=2000, shape =(2,), dtype=np.float32),
                self.device,
                handle_timeout_termination=False,
            )

        env = env_risk(**(self.env_config | {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}))
        env.reset(42)
        
        self.train_loop_init()
        
        for iteration in range(1, self.num_iterations+1):

            obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
            actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
            action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
            current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
            current_phase = torch.zeros((self.num_steps,1)).to(self.device)
            current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
            logprobs = torch.zeros((self.num_steps, )).to(self.device)
            rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            #values = torch.zeros((self.num_steps,  )).to(self.device)
            episodes = torch.ones((self.num_steps, )).to(self.device)*-1
            t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)








            
            rb = self.sample(
                                env,iteration,
                                obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,values
                                ,episodes
                                ,t_next,
                                rb
                 
                            )
            
            self.train(rb,iteration)


            
            
            if self.global_step%100 ==0:
                SPS = int(self.global_step / (time.time() - self.start_time))
                print("SPS:", SPS)       
                self.writer.add_scalar("charts/SPS", SPS, self.global_step)
        

            self.save_models()
    
    def sample(self,env,iteration,obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,values
                                ,episodes
                                ,t_next,
                                rb):
        

        
        
        with torch.no_grad():
            # sample `worker_steps` from each worker
            #there are no worker steps... rather there are full episodes

            step = 0
            fault_condition = False
            clear_output(wait=True)
            phase = 0
        
            
            for episode in range(self.num_episodes):#num_episodes):
                
                total_rewards = {i:0 for i in env.possible_agents} #i can report this
                action=1
                
                if fault_condition:
                    env = env_risk(**(self.env_config | {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False})
                                      )#game.env(render_mode=None)

                curren_epi = episode + (iteration-1)*self.num_episodes
                env.reset(curren_epi) #for riplication
                
                fault_condition = False
                step_count = 0
                bad_move_count = 0
                bad_move_phase_count = {i:0 for i in env.phases}
                move_count =  {i:0 for i in env.phases}
                is_draw = 0
                
                draw_territory_count = 0
                is_third = 0

                for agent in env.agent_iter():
                    e_t = env.terminations
                    if sum(e_t.values()) <2:
                        observation, reward, termination, truncation, info = env.last()
        
                        observation['observation'] =  self.obs_converter(
                                                        torch.tensor(
                                                            observation['observation']
                                                        ).to(self.device,dtype=torch.float32))
                        
                        episodes[step] = curren_epi
                        obs[step] = observation['observation']#torch.Tensor(observation['observation']).to(self.device) #sould i not add it .... meaning this is the last observation after the player dies
                        action_masks[step] = torch.Tensor(observation['action_mask']).to(self.device)
                        
                        #curr_agent = agent#int(agent[-1])
                        current_agent[step] = curr_agent = agent
                        current_phase[step] = phase = env.phase_selection
                        phase_mapping = self.map_agent_phase_hot(phase,num_classes = self.total_phases).float()
                        
                        curr_agent_mapping = self.map_agent_phase_hot(int(curr_agent)-1,
                                                                      num_classes = self.total_agents 
                                                                     ).float()
                        
                        current_troops_count[step] = torch.Tensor([env.board.agents[i].bucket for i in env.possible_agents]).to(self.device)
                    

                        model_in = torch.Tensor(torch.hstack((observation['observation'].reshape(-1),torch.tensor(observation['action_mask'].reshape(-1)).to(self.device),
                                           phase_mapping,
                                            curr_agent_mapping,
                                           torch.tensor([env.board.agents[curr_agent].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
                                                ).float()
                        
                        #if e_t[curr_agent]:
                            #print('heeee')
                            
                        if termination or truncation:
                            
                            action = None

                            act = self.actor(torch.Tensor(model_in).to(self.device))
                            #act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                            #values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                                #have taken at this point and what would have been its value
                            actions[step] = act #even after going what would have been
                            #logprobs[step] = logprob        
                        else:
                            mask = observation["action_mask"]
                            if (self.global_step < self.args.learning_starts) or (
                                np.random.rand() > min(
                                                ((curren_epi)/((self.num_iterations*self.num_episodes)/10))
                                                , 0.95)
                                                ) or (
                                agent != self.the_hero_agent):
        
                                
                                action = env.action_space(agent).sample()
                                part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                                action = torch.Tensor([[[part_0],[np.around(action[1],2)]]]).to(self.device)
                                action = action[:,:,0]
                            else:
                                
                                action = self.actor(torch.Tensor(model_in).to(self.device))
                            actions[step] = action
        
                            if not observation['action_mask'][action[:,0].long()]: 
                                fault_condition =True
                                #self.faulting_player = agent
        
                                if self.the_hero_agent == curr_agent:
                                    bad_move_count+=1
                                    bad_move_phase_count[int(current_phase[step][0])]+=1  # when is the where_is_it_performing_bad_really
                                    #print('here',agent, action, observation['action_mask'])
                            
        
                            if self.the_hero_agent == curr_agent:
                                move_count[int(current_phase[step][0])]+=1        
        
        
                        #print('here',agent, action)
                        if action != None :
                            act_2 = action.detach().cpu().numpy()[0]#list([action.detach().cpu().numpy()[0][0], max(action.detach().cpu().numpy()[0][1],0.1) ])
                            act_2 = list([act_2[0], max(act_2[1],0.001) ])
                        else:
                            act_2 = action
                            
                        env.step(act_2 if action != None else None)        
        
        
                        if action == None:
                            print('heeee')
                            rewards[step] = np.zeros(self.total_agents) # should i keep it -1? .... hm i dont think so .
                            dones[step] = np.zeros(self.total_agents) # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                            # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                            #values[step] = 
                        else:
        
                            rewards_2[step] = torch.Tensor([env.curr_rewards[i] for i in env.possible_agents]).to(self.device)
                            curr_reward_list =  env.curr_rewards
                            if (step_count == (self.episode_time_lim-1)):
                                curr_reward_list = {i:-100 for i in env.possible_agents }
        
                            for i in env.possible_agents:
                                if i != curr_agent:
                                    self.gamma_t[i]+=1
                                else:
                                    self.gamma_t[i] =0
        
                                if (step_count == (self.episode_time_lim-1)):
                                    cr_rew = -100
                                    term = True
                                else:
                                    cr_rew = env.curr_rewards[i]
                                    term = env.terminations[i]

                                next_step_ = step-self.gamma_t[i]
                                rewards[next_step_,i-1] += (self.args.gamma**self.gamma_t[i])*cr_rew
                                t_next[next_step_,i-1] = self.gamma_t[i]
                                dones[next_step_,i-1] = torch.Tensor([term]).to(self.device) #so the panetly has to be added but attributions is really difficult
        
                        #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                        
                        if sum(curr_reward_list.values()) == -300:
                            #print('here')
                            is_draw=1
        
                        
                        for age_i in env.possible_agents:
                            
                            total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                                    
                        
                        step +=1
                        self.global_step+=1
        
                    else:
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        break    
                
        
        
        
                    step_count+=1
                    
                    if (self.global_step == self.num_steps) :# or (fault_condition and (fa ulting_player != agent) and (len(env.agents)==0)):
                        break
                    elif (step_count == self.episode_time_lim):
                        break
                        
                #print(rewards[step-2])
                if self.global_step == self.num_steps:
                    break 
                
                position = 3
                for k_,(i_,j_) in enumerate(sorted([(j_,i_) for i_,j_ in total_rewards.items()],reverse=True) 
                      ):
                    if j_==self.the_hero_agent:
                        position = k_+1

                cur_epi_list = (episodes == curren_epi)
                
                self.write_exploring(is_draw,position,curren_epi,step,total_rewards,bad_move_count
                                    ,bad_move_phase_count,
                                    move_count,observation,env,cur_epi_list)


            t_range = torch.Tensor(np.arange(0,step)).to(self.device,dtype=torch.int)
            hero_steps = [current_agent == self.the_hero_agent][0][:,0][:step]
            
            next_indecies = (t_next[:step,self.the_hero_agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
        
            selected_t_next = t_next[:,self.the_hero_agent-1,None]#t_next[:step,0,None][hero_steps]
            infos = [dir({})]*step #t_next[:step,0]
            b_obs_a = torch.concat(( obs.reshape(-1,np.prod(self.ob_space_shape)) ,
                                    action_masks.reshape(-1,np.prod(self.action_mask_shape)),
                                    self.map_agent_phase_vector(current_agent,num_classes=self.total_agents+1)[:,1:],
                                    
                                    #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                                    self.map_agent_phase_vector(current_phase,num_classes=self.total_phases),
                                    current_troops_count[:,self.the_hero_agent-1,None],
                                    selected_t_next
                                   ),axis =1)


            
            for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
               actions[:step][hero_steps].cpu().to(dtype=torch.float32), rewards[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), 
               dones[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), infos):
                rb.add(*i)







        avg_episode_length = torch.mean(torch.tensor(
                            [(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()]).float())#np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
        
        self.writer.add_scalar("charts/avg_episodic_length", avg_episode_length, self.global_step)


        
        return rb


    def write_exploring(self,is_draw,position,curren_epi,step,total_rewards,bad_move_count
                        ,bad_move_phase_count,
                        move_count,observation,env,cur_epi_list):

        if is_draw:
            self.draw_count +=1
            if position ==3:
                self.third_count_draw  +=1
                
            #    draw_territory_count = 0
            #else:
            draw_territory_count = int(observation['observation'][:,self.the_hero_agent].sum())
            self.writer.add_scalar("win_charts/1st_position_prop_draw",int(position==1),self.draw_count)#(curren_epi+1))
            self.writer.add_scalar("win_charts/2nd_position_prop_draw",int(position==2),self.draw_count)#(curren_epi+1))
            self.writer.add_scalar("win_charts/3rd_position_prop_draw",int(position==3),self.draw_count)#(curren_epi+1))
            
            self.writer.add_scalar("win_charts/3rd_position_all_prop",int(position ==3),(curren_epi+1))
            
            self.writer.add_scalar("win_charts/draw_count",self.draw_count,self.global_step)
            self.writer.add_scalar("win_charts/draw",1,(curren_epi+1))
            self.writer.add_scalar("win_charts/draw_to_total_count",self.draw_count/(curren_epi +0.000001),self.global_step)
            
            self.writer.add_scalar("win_charts/draw_territory_count",draw_territory_count,self.draw_count)#self.global_step)

        
            self.writer.add_scalar("win_charts/third_place_in_draw",self.third_count_draw,self.draw_count)#self.global_step)
            self.writer.add_scalar("win_charts/third_place_in_draw_ratio",self.third_count_draw/self.draw_count,self.draw_count)#self.global_step)
        else:
            
            self.first_count += (position == 1)
            self.second_count += (position == 2)
            self.third_count += (position == 3)
            self.writer.add_scalar("win_charts/1st_position_prop",int(position==1),(curren_epi+1))
            self.writer.add_scalar("win_charts/2nd_position_prop",int(position==2),(curren_epi+1))
            self.writer.add_scalar("win_charts/3rd_position_prop",int(position==3),(curren_epi+1))
            self.writer.add_scalar("win_charts/3rd_position_all_prop",int(position==3),(curren_epi+1))
            self.writer.add_scalar("win_charts/draw",0,(curren_epi+1))
            
            self.writer.add_scalar("win_charts/1st_position",self.first_count,(curren_epi+1))#global_step)

        
            self.writer.add_scalar("win_charts/2nd_position",self.second_count,(curren_epi+1))#global_step)
            self.writer.add_scalar("win_charts/3rd_position",self.third_count,(curren_epi+1))#global_step)  
            non_draw_count =(curren_epi-self.draw_count+0.000001)
            self.writer.add_scalar("win_charts/1st_position_to_total_terminated",self.first_count/non_draw_count,(curren_epi+1))#global_step)

        
            self.writer.add_scalar("win_charts/2nd_position_to_total_terminated",self.second_count/non_draw_count,(curren_epi+1))#global_step)
            self.writer.add_scalar("win_charts/3rd_position_to_total_terminated",self.third_count/non_draw_count,(curren_epi+1))#global_step)             

        self.writer.add_scalar("win_charts/3rd_position_to_total",(self.third_count+self.third_count_draw)/(curren_epi +0.00001 ),(curren_epi+1))#global_step)

        
        self.writer.add_scalar("charts/epsilon",(curren_epi/((self.num_iterations*self.num_episodes)/10)),self.global_step)
        self.writer.add_scalar("charts/avg_per_epi_total_reward", np.mean(list(total_rewards.values())), self.global_step)
        self.writer.add_scalar("new_charts/bad_move_count_per_episode",bad_move_count,self.global_step)
        
        self.writer.add_scalar("new_charts/bad_move_count_position_per_episode",bad_move_phase_count[0],self.global_step)
        self.writer.add_scalar("new_charts/bad_move_count_attack_per_episode",bad_move_phase_count[1],self.global_step)
        self.writer.add_scalar("new_charts/bad_move_count_fortify_per_episode",bad_move_phase_count[2],self.global_step)

        self.writer.add_scalar("new_charts/total_moves",sum(move_count.values()),self.global_step)


        self.writer.add_scalar("new_charts/bad_move_to_step_count_per_episode",bad_move_count/(sum(move_count.values())+1),self.global_step)

        self.writer.add_scalar("new_charts/bad_move_to_step_position_per_episode",bad_move_phase_count[0]/( move_count[0]+1),self.global_step)
        self.writer.add_scalar("new_charts/bad_move_to_step_attack_per_episode",bad_move_phase_count[1]/( move_count[1]+1),self.global_step)
        self.writer.add_scalar("new_charts/bad_move_to_step_fortify_per_episode",bad_move_phase_count[2]/( move_count[2]+1),self.global_step)


        #values_total = {i:0 for i in self.env.possible_agents}
        

        
        self.writer.add_scalar("charts/episodic_length", cur_epi_list[:step].sum(), self.global_step)
        
        for i in env.possible_agents:
            #cur_index = torch.where((current_agent[:,0] == i) &( cur_epi_list ))[0]

            #values_total[i] = values[cur_index].mean()
            #writer.add_scalar("charts/mean_value_per_epi_agent_"+str(i), values_total[i], global_step)
            
            self.writer.add_scalar("charts/total_reward_per_epi_agent_"+str(i), total_rewards[i], self.global_step)

    
    
    def train(self,rb,iteration):
        
        for epoch in range(self.args.update_epochs):
            
            if self.global_step > self.args.learning_starts:
                data = rb.sample(self.args.batch_size)
                with torch.no_grad():
                    collected_t_next = data.next_observations[:,-1]
                    next_state_actions = self.target_actor(data.next_observations[:,:-1])
                    qf1_next_target = self.qf1_target(data.next_observations[:,:-1], next_state_actions)
    
                    
                    next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (self.args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
            
                qf1_a_values = self.qf1(data.observations[:,:-1], data.actions).view(-1)
                qf1_loss = nn.functional.mse_loss(qf1_a_values, next_q_value)
                
                # optimize the model
                self.q_optimizer.zero_grad()
                qf1_loss.backward()
                self.q_optimizer.step()
                
                #if global_step % args.policy_frequency == 0:
                actor_loss = -self.qf1(data.observations[:,:-1], self.actor(data.observations[:,:-1])).mean()
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()
            
                # update the target network
                for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
                    target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
                for param, target_param in zip(self.qf1.parameters(), self.qf1_target.parameters()):
                    target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
    
                ind_epoch = epoch + (iteration-1)*self.args.update_epochs
                
                self.writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), ind_epoch)
                
                self.writer.add_scalar("losses/qf1_loss", qf1_loss.item(), ind_epoch)
                self.writer.add_scalar("losses/actor_loss", actor_loss.item(), ind_epoch)
                #print("SPS:", int(global_step / (time.time() - start_time)))
                
        
    def save_models(self):
        newpath = r'./models/' + self.run_name 
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.actor.state_dict(), newpath+"/actor.pt")
        torch.save(self.qf1.state_dict(), newpath+"/qf1.pt")
        torch.save(self.qf1_target.state_dict(), newpath+"/qf1_target.pt")
        torch.save(self.target_actor.state_dict(), newpath+"/target_actor.pt")

        
        




In [27]:
T = Trainer(Args)

In [ ]:
T.run_training_loop()

done: {1: True, 2: True, 3: True} ,total_reward: {1: 105.05000000000004, 2: -202.07, 3: -105.32} ,iteration: 300 ,episode: 0
done: {1: True, 2: True, 3: True} ,total_reward: {1: 104.68000000000005, 2: -201.04000000000002, 3: -102.46999999999998} ,iteration: 300 ,episode: 1


In [2]:
T.device

NameError: name 'T' is not defined